In [0]:
from pyspark.sql.functions import explode, col, trim, monotonically_increasing_id, countDistinct, row_number, coalesce, lit, length, split, avg, when, uuid, regexp_extract, regexp_replace
from pyspark.sql.window import Window

# Set catalog and schema

In [0]:
spark.sql("USE CATALOG `bigdata-and-bi`")
spark.sql("USE SCHEMA gold")

print("Using catalog: bigdata-and-bi, schema: gold")

✅ Using catalog: bigdata-and-bi, schema: gold


# Load cleaned data from silver schema

In [0]:
items_df = spark.table("`bigdata-and-bi`.silver.items_clean")
reviews_df = spark.table("`bigdata-and-bi`.silver.reviews_clean")

print("✅ Loaded input tables:")
print(f"Items: {items_df.count()} rows")
print(f"Reviews: {reviews_df.count()} rows")

✅ Loaded input tables:
Items: 4448180 rows
Reviews: 27753205 rows


# Extract publisher in details to separate column

In [0]:
items_df = items_df.withColumn(
    "publisher_raw",
    col("details")["Publisher"]
)

items_df = items_df.withColumn(
    "publisher",
    coalesce(
        trim(
            regexp_extract(col("publisher_raw"), r"^([^;(]+)", 1)
        ),
        lit("Unknown")
    )
)

# Extract contributor to separate column

In [0]:
# Normalize nulls
items_df = items_df.withColumn(
    "contributor_clean",
    when(col("store").isNull(), lit("Unknown")).otherwise(col("store"))
)

# Split into array
items_df = items_df.withColumn(
    "contributor_array",
    split(
        col("contributor_clean"),
        r",\s*(?![^()]*\))"   # split only commas NOT inside (...)
    )
)

# Explode into rows and remove role inside parentheses
contributors_exploded = (
    items_df
    .withColumn("contributor_entry", explode(col("contributor_array")))
    .withColumn(
        "contributor_name",
        trim(regexp_extract(col("contributor_entry"), r"^(.+?)\s*\(", 1))
    )
)

contributors_exploded = contributors_exploded.withColumn(
"contributor_name",
trim(regexp_replace(col("contributor_name"), r"^.*?Edition +", ""))
)


# Handle cases with no parentheses (like "Unknown" or names without roles)
contributors_exploded = contributors_exploded.withColumn(
    "contributor_name",
    when(col("contributor_name") == "", trim(col("contributor_entry")))
    .otherwise(col("contributor_name"))
)

contributors_exploded = contributors_exploded.withColumn(
    "space_count",
    length(col("contributor_name")) - length(regexp_replace(col("contributor_name"), " ", ""))
)

# Filter:
# 1) name length <= 65
# 2) space_count <= 5 (adjustable)
contributors_exploded = contributors_exploded.filter(
    (length(col("contributor_name")) <= 65) &
    (col("space_count") <= 5)
)

# Drop helper column
contributors_exploded = contributors_exploded.drop("space_count")



# Create dim_contributor table

In [0]:
dim_contributor = (
    contributors_exploded
    .select("contributor_name")
    .dropDuplicates()
    .withColumn("contributor_id", uuid())
    .select("contributor_id", "contributor_name")
)
dim_contributor.write.mode("overwrite").saveAsTable("gold.dim_contributor")

distinct_count_df = contributors_exploded.select(countDistinct("contributor_name"))


# Create bridge_item_contributor table

In [0]:
dim_contributor = spark.table("`bigdata-and-bi`.gold.dim_contributor")

bridge_item_contributor = (
    contributors_exploded
    .join(dim_contributor, on="contributor_name", how="inner")
    .select("parent_asin", "contributor_id")
    .dropDuplicates(["parent_asin", "contributor_id"])
)
bridge_item_contributor.write.mode("overwrite").saveAsTable("gold.bridge_item_contributor")

# Create dim_avg_rating table

In [0]:
dim_avg_rating = (
    items_df
    .select("average_rating")
    .dropna(subset=["average_rating"])
    .dropDuplicates(["average_rating"])
    .withColumn("avg_rating_id", uuid())
    .select("avg_rating_id", "average_rating")  # reorder
)
dim_avg_rating.write.mode("overwrite").saveAsTable("gold.dim_avg_rating")

# Create dim_category table

In [0]:
# explode categories array
exploded_categories = (
    items_df
    .select(
        explode("categories").alias("category")
    )
    .withColumn("category", trim(col("category")))
    .dropna()
)

# dedupe categories
dim_category = (
    exploded_categories
    .dropDuplicates(["category"])
    .withColumn("category_id", uuid())
    .select("category_id", "category")  # reorder
)
dim_category.write.mode("overwrite").saveAsTable("gold.dim_category")

# Create bridge_item_category table

In [0]:
# Re-explode categories with product_id
item_categories = (
    items_df
    .select(
        col("parent_asin"),
        explode("categories").alias("category")
    )
    .withColumn("category", trim(col("category")))
)

# Join to dim_category to map to category_id
dim_category = spark.table("`bigdata-and-bi`.gold.dim_category")
bridge_item_category = (
    item_categories
    .join(dim_category, on="category", how="inner")
    .select("parent_asin", "category_id")
    .dropDuplicates(["parent_asin", "category_id"]) 
)
bridge_item_category.write.mode("overwrite").saveAsTable("gold.bridge_item_category")


# Create dim_main_category

In [0]:
dim_main_category = (
    items_df
    # fill null category with "Unknown"
    .withColumn(
        "main_category",
        when(col("main_category").isNull(), "Unknown").otherwise(col("main_category"))
    )
    .select("main_category")
    .dropDuplicates(["main_category"])
    .withColumn("main_category_id", uuid())
    .select("main_category_id", "main_category")  # reorder
)
dim_main_category.write.mode("overwrite").saveAsTable("gold.dim_main_category")

# Create dim_publisher table

In [0]:
dim_publisher = (
    items_df
    .select("publisher")
    .dropDuplicates()
    .withColumn("publisher_id", uuid())
    .select("publisher_id", "publisher")
)
dim_publisher.write.mode("overwrite").saveAsTable("gold.dim_publisher")


# Create dim_review table

In [0]:
# Count reviews per product
review_counts = (
    reviews_df
    .groupBy("parent_asin")
    .count()
    .withColumnRenamed("count", "total_reviews")
)

# Use items_df (or fact_item) to include ALL products
dim_review = (
    items_df
    .select(col("parent_asin"))
    .join(review_counts, on="parent_asin", how="left")
    .withColumn("total_reviews", coalesce(col("total_reviews"), lit(0)))
    .select("parent_asin", "total_reviews")  # reorder
)


# Create fact_item table

In [0]:
# compute median (approx because Spark)
median_price = items_df.approxQuantile("price", [0.5], 0.01)[0]
dim_avg_rating = spark.table("`bigdata-and-bi`.gold.dim_avg_rating")
dim_main_category = spark.table("`bigdata-and-bi`.gold.dim_main_category")
dim_publisher = spark.table("`bigdata-and-bi`.gold.dim_publisher")


fact_item = (
    items_df
    .select("parent_asin", "title", "average_rating", "rating_number", "publisher",
            coalesce(col("store"), lit("Unknown")).alias("store"), 
            coalesce(col("price"), lit(median_price)).alias("price"), 
            coalesce(col("main_category"), lit("Unknown")).alias("main_category")
    )
    .dropna(subset=["parent_asin"])
    .dropDuplicates(["parent_asin"])
    .join(dim_avg_rating, on="average_rating", how="left")
    .join(dim_main_category, on="main_category", how="left")
    .join(dim_review.select("parent_asin", "total_reviews"), on="parent_asin", how="left")
    .join(dim_publisher, on="publisher", how="left")
    .select(
        "parent_asin",
        "title",
        "price",
        "rating_number",
        "total_reviews",
        "avg_rating_id",
        "main_category_id",
        "publisher_id"
    )
)

fact_item.write.mode("overwrite").saveAsTable("gold.fact_item")
print("✅ All schema tables saved successfully!")

✅ All schema tables saved successfully!
